In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from itertools import cycle 
a = 'ì..ìì...................'
b = '...................ì..ìì'
print(a)
print(b)



def shift(s, d):
    if d == +1:
        if s[-1] == 'ì':
            return s
        else:
            shf = s[len(s)-1:] + s[:len(s)-1]
            return shf
    if d == -1:
        if s[0] == 'ì':
            return s
        else:
            shf = s[1:] + s[:1] 
            return shf
il = 0
ir = 0
dl = None
dr = None
bb1 = b
bb2 = b
while bb1 != a:
    il += 1
    nbb1 = shift(bb1, 1)
    print(nbb1)
    if nbb1 == a: 
        dl = il
        break
    if bb1 == nbb1: 
        print('wall')
        break
    else: bb1 = nbb1
     
while bb2 != a:
    ir += 1
    nbb2 = shift(bb2, -1)
    print(nbb2)
    if nbb2 == a:
        dr = ir 
        break
    if bb2 == nbb2: 
        print('wall')
        break
    else: bb2 = nbb2
    
print(dl, dr)

dist = dl if dl != None else dr 

print('register', dist)
print(a+'+'+b[::-1])

ì..ìì...................
...................ì..ìì
...................ì..ìì
wall
..................ì..ìì.
.................ì..ìì..
................ì..ìì...
...............ì..ìì....
..............ì..ìì.....
.............ì..ìì......
............ì..ìì.......
...........ì..ìì........
..........ì..ìì.........
.........ì..ìì..........
........ì..ìì...........
.......ì..ìì............
......ì..ìì.............
.....ì..ìì..............
....ì..ìì...............
...ì..ìì................
..ì..ìì.................
.ì..ìì..................
ì..ìì...................
None 19
register 19
ì..ìì...................+ìì..ì...................


In [3]:
import sys
sys.path.append('..')

import numpy as np

from hdna import *

In [4]:
stru = Structure('...(..(((.(....+......).)))..).')
print(stru.register)
stru.tails_l,stru.tails_r,stru.bulk

True ^ True

-2


False

In [5]:
struta = Structure(stru.table, fromtable=True)
print(struta.str == stru.str)
print(struta.left == stru.left)
print(struta.right == stru.right)


True
True
True


In [6]:
import networkx as nx 
M = Model(standard=True)
M.setgeometry(theta=90, phi=180)
# A = Strand.random(M, 18)
# print(A.sequence)
A = Strand(M, 'AAAAAAAAAAAAAA')
B = A.complementary()

In [31]:
K = Kinetics(M, A, B)
N = Kinetwork(M, A, B)
# print(A.sequence)
N.save_graph('.')

distance 4.42e-07
r3 6.76e-08
r4 6.76e-08
overlap2 1.7555097586382238e-20
0.07142857142857142
0.5
pseudoknot
(((((((((((((.+))))))))))))). ((((((((((((((+))))))))))))))
collision 1.568e+07
1.458e+06 1.683e+05
fres -15.172323968066781 -16.417044264697445
zeta 6.669241112884172e-11 0.9999999999333076

distance 4.08e-07
r3 9.560083681642121e-08
r4 9.560083681642121e-08
overlap2 9.725887637096607e-21
0.14285714285714285
0.25
pseudoknot
((((((((((((..+)))))))))))).. ((((((((((((((+))))))))))))))
collision 1.568e+07
1.276e+07 1.690e+05
fres -13.927603671436119 -16.417044264697445
zeta 4.394769798576078e-09 0.9999999956052301

distance 3.74e-07
r3 1.170866345916561e-07
r4 1.170866345916561e-07
overlap2 4.311613567165481e-21
0.21428571428571427
0.16666666666666666
pseudoknot
(((((((((((...+)))))))))))... ((((((((((((((+))))))))))))))
collision 1.568e+07
1.123e+08 1.697e+05
fres -12.682883374805456 -16.417044264697445
zeta 2.895981091512626e-07 0.9999997104018908

distance 3.4000000000000003e-0

In [8]:
from itertools import permutations, combinations, pairwise

nodes = list(   N.filternodes('pairs', lambda x: x == 1, 
                N.filternodes('dpxdist', lambda x: x == -1, N.DG)).nodes())
nodes = [Structure(node) for node in nodes]
classifica = []
for packs in range(len(nodes)):
    for combo in combinations(nodes,packs):
        empty = Structure.empty(18)
        for node in combo:
            empty = [e1 or e2 for e1, e2 in zip(node.table, empty.table)]
            empty = Structure(empty, fromtable=True)
            compx = Complex(M, A, B.invert, state='backfray', structure=empty.str)
            classifica.append((empty.str, compx.G))

In [9]:
import pandas as pd 

pd.DataFrame(classifica).drop_duplicates().sort_values(1)

,0,1
53234,..((((((((((((+.)))))))))))).,-13.927604
53090,.((((((((((((.+..)))))))))))),-13.927604
53078,...(((((((((((+.)))))))))))..,-12.682883
52231,.(((((((((((..+...))))))))))),-12.682883
52957,..(((((((((((.+..))))))))))).,-12.682883
...,...,...
1530,.(..(...(....(+.)....)...)..),11.067705
1642,.(...(..(....(+.)....)..)...),11.067705
1546,.(..(....(...(+.)...)....)..),11.067705
1742,.(....(..(...(+.)...)..)....),11.067705


In [10]:
from itertools import combinations
edges = N.filteredges('state', lambda x: x in ['zipping', 'backfray'], N.DG)
fres = []
for e in edges.edges():
    fres.append(abs(N.DG.nodes[e[0]]['fre'] - N.DG.nodes[e[1]]['fre']))
np.mean(fres)

1.2447202966306623

In [19]:
N.connect_slidings()

inchworm
(((((((((((((.+))))))))))))). ((((((((((((((+))))))))))))))
kbulged  4.000e+07
kbulging 4.000e+07
inchworm 4.000e+07

inchworm
(((((((((((((.+))))))))))))). ((((((((((((((+))))))))))))))
kbulged  4.000e+07
kbulging 4.000e+07
inchworm 4.000e+07

inchworm
(((((((((((((.+))))))))))))). ((((((((((((((+))))))))))))))
kbulged  4.000e+07
kbulging 4.000e+07
inchworm 4.000e+07

inchworm
((((((((((((..+)))))))))))).. ((((((((((((((+))))))))))))))
kbulged  2.000e+07
kbulging 2.000e+07
inchworm 2.000e+07

inchworm
((((((((((((..+)))))))))))).. ((((((((((((((+))))))))))))))
kbulged  2.000e+07
kbulging 2.000e+07
inchworm 2.000e+07

inchworm
(((((((((((...+)))))))))))... ((((((((((((((+))))))))))))))
kbulged  1.333e+07
kbulging 1.333e+07
inchworm 1.333e+07

inchworm
(((((((((((...+)))))))))))... ((((((((((((((+))))))))))))))
kbulged  1.333e+07
kbulging 1.333e+07
inchworm 1.333e+07

inchworm
((((((((((....+)))))))))).... ((((((((((((((+))))))))))))))
kbulged  1.000e+07
kbulging 1.000e+07
inch

In [12]:
nodz = list(N.filternodes('state', lambda x: x == 'on_nucleation', N.DG).nodes())

objs = [N.DG.nodes[node]['obj'] for node in nodz]

In [13]:
for ob in objs:
    print(ob.structure)

(.............+.............)
.(............+............).
..(...........+...........)..
...(..........+..........)...
....(.........+.........)....
.....(........+........).....
......(.......+.......)......
.......(......+......).......
........(.....+.....)........
.........(....+....).........
..........(...+...)..........
...........(..+..)...........
............(.+.)............
.............(+).............


In [14]:
def get_ix(string, char):
    indices = []
    for i, e in enumerate(list(string)):
        if e == char:
            indices.append(i)
    return indices 

In [15]:
on = 0
print(objs[on].structure)
l = objs[on].structure.split('+')[0]
r = objs[on].structure.split('+')[1][::-1]
ixl = get_ix(l,'(')[0]
ixr = get_ix(r,')')[0]

print(objs[on].s1.sdist[ixl])
objs[on].s1.sdist[ixl] == objs[on].s1.sdist[ixr]


(.............+.............)
0


True

In [16]:
np.ceil(2*SXGEO.PERSISTENCE/SXGEO.MONODIST)

7.0

In [17]:
print(f'{K.ksphere_sano():.3e}')

-6.349e+14


In [18]:
bottomheignt = 2*SXGEO.PERSISTENCE
topgyradius  = K.gyradiuses(N=4)
kc = K.ksphere_sano(a=topgyradius, badd=(bottomheignt))
print(f'{kc:.3e}')
kc = K.ksphere_sano(SXGEO.CY_RADIUS)
print(f'{kc:.3e}')
print()
print(SXGEO.CY_RADIUS)
print(K.gr1)

-5.260e+14
-4.984e+14

1e-07
2.5293603934591843e-07
